<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## Data Processing Workflow

In this workflow within PyCCAPT, we can crop the data, do the voltage and bowl calibration, calculate the 3d reconstruction, and do the ranging.
</div>

In [1]:
# Activate intractive functionality of matplotlib
%matplotlib ipympl
# Activate auto reload 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
# import libraries
import os
import numpy as np
import pandas as pd
import subprocess
from ipywidgets import widgets
from IPython.display import display
from ipywidgets import fixed, interact_manual
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

# Local module and scripts
from pyccapt.calibration.calibration import widgets as wd
from pyccapt.calibration.data_tools import data_tools, data_loadcrop
from pyccapt.calibration.tutorials.tutorials_helpers import helper_calibration
from pyccapt.calibration.tutorials.tutorials_helpers import helper_data_loader
from pyccapt.calibration.tutorials.tutorials_helpers import helper_temporal_crop
from pyccapt.calibration.tutorials.tutorials_helpers import helper_special_crop
from pyccapt.calibration.tutorials.tutorials_helpers import helper_t_0_tune
from pyccapt.calibration.tutorials.tutorials_helpers import helper_mc_plot
from pyccapt.calibration.tutorials.tutorials_helpers import helper_3d_reconstruction
from pyccapt.calibration.tutorials.tutorials_helpers import helper_ion_selection
from pyccapt.calibration.tutorials.tutorials_helpers import helper_visualization
from pyccapt.calibration.tutorials.tutorials_helpers import helper_ion_list
from pyccapt.calibration.calibration import ion_selection
from pyccapt.calibration.calibration import share_variables
from pyccapt.calibration.leap_tools.leap_tools import read_rrng

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
In case of recieving the error about pytable library, you have to install the pytables library with conda command. to do that you can open a new cell and copy the line below in it. Then just run it like other cells. The pytables library will be innstalled.
    
`!conda install --yes --prefix {sys.prefix} pytables`
</div>


<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
In the cell below we create variable object. This object is used in many of the functions to share the data between functions in as easy way. 
</div>


In [2]:
variables = share_variables.Variables()

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
    
By clicking on the button below, you can select the dataset file you want to work with. The dataset file can be in various formats, including HDF5, EPOS, POS, ATO, and CSV. The cropped data will be saved in the same directory as the original dataset file in a new directory nammed load_crop. The name of the cropped dataset file will be the same as the original dataset file. The figures will be saved in the same directory as the dataset file.
</div>

In [3]:
button = widgets.Button(description='load dataset')

@button.on_click
def open_file_on_click_r(b):
    global dataset_path
    folder_path = variables.last_directory
    script = '..//..//data_tools//run_dataset_path_qt.py'
    cmd = f"python {script} {folder_path}"
    result = subprocess.run(cmd, capture_output=True, text=True, shell=True)
    dataset_path = result.stdout.strip()
    variables.last_directory = dataset_path

button

Button(description='load dataset', style=ButtonStyle())

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## ROI Selection and Data Cropping
    

</div>



<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

    
From the dropdown lists below, you can select the instrument specifications of the dataset. The instrument specifications are the same as the ones used for the calibration process. Data mode is specify the dataset structure. The dataset can be in raw or calibrated mode. The flight path length is the distance between the sample and the detector. The t0 is the time of flight of the ions with the lowest mass-to-charge ratio. The maximum mass-to-charge ratio is the maximum mass-to-charge ratio of tat you want to plot. You can also change it in te related cells. The detector diameter is the diameter of the detector.
</div>


In [4]:
# create an object for selection of instrument specifications of the dataset
tdc, pulse_mode, flight_path_length, t0, max_mc, det_diam = wd.dataset_instrument_specification_selection()

# Display lists and comboboxes to selected instrument specifications
display(tdc, pulse_mode, flight_path_length, t0, det_diam, max_mc)

Dropdown(description='Data mode:', options=('pyccapt', 'leap_epos', 'leap_pos', 'leap_apt', 'ato_v6'), value='…

Dropdown(description='Pulse mode:', options=('voltage', 'laser'), value='voltage')

FloatText(value=110.0, description='Flight path length:')

FloatText(value=38.0, description='t0:')

FloatText(value=80.0, description='Detector diameter:')

FloatText(value=400.0, description='Max mc:')

In [5]:
helper_data_loader.load_data(dataset_path, max_mc.value, flight_path_length.value, pulse_mode.value, tdc.value, variables)
data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)
display(variables.data)
display(variables.range_data)

The maximum possible TOF is: 5010 ns
The data will be saved on the path: T:/Monajem/APT_data/2382_Jan-10-2025_15-12_NiC9_Al/2382_Jan-10-2025_15-12_NiC9_Al/data_processing/
The dataset name after saving is: 2382_Jan-10-2025_15-12_NiC9_Al
The figures will be saved on the path: T:/Monajem/APT_data/2382_Jan-10-2025_15-12_NiC9_Al/2382_Jan-10-2025_15-12_NiC9_Al/data_processing/
{'apt': ['experiment_chamber_vacuum', 'id', 'num_events', 'num_raw_signals', 'temperature', 'timestamps'], 'dld': ['high_voltage', 'laser_pulse', 'start_counter', 't', 'voltage_pulse', 'x', 'y'], 'tdc': ['channel', 'high_voltage', 'laser_pulse', 'start_counter', 'time_data', 'voltage_pulse']}
The data is loaded in raw mode
The number of data that is removed: 43356
Total number of Ions: 23050746
The maximum possible time of flight is: 5010


,x (nm),y (nm),z (nm),mc (Da),mc_uc (Da),high_voltage (V),pulse,t (ns),t_c (ns),x_det (cm),y_det (cm),delta_p,multi,start_counter
0,0.0,0.0,0.0,0.0,0.0,500.00,328.000,1560.517326,0.0,1.211429,0.786939,0.0,0.0,12877
1,0.0,0.0,0.0,0.0,0.0,500.00,328.000,3862.638198,0.0,1.315918,-0.538776,0.0,0.0,12897
2,0.0,0.0,0.0,0.0,0.0,500.00,328.000,4725.038556,0.0,1.795918,-0.884898,0.0,0.0,328
3,0.0,0.0,0.0,0.0,0.0,500.00,328.000,4286.572326,0.0,1.126531,-1.453061,0.0,0.0,5238
4,0.0,0.0,0.0,0.0,0.0,500.00,328.000,2820.023316,0.0,-3.983673,-0.153469,0.0,0.0,6942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23050741,0.0,0.0,0.0,0.0,0.0,4200.38,840.076,682.816770,0.0,2.390204,-2.259592,0.0,0.0,19720
23050742,0.0,0.0,0.0,0.0,0.0,4200.38,840.076,4706.048754,0.0,1.776327,-1.191837,0.0,0.0,19721
23050743,0.0,0.0,0.0,0.0,0.0,4200.38,840.076,3621.936114,0.0,-2.279184,1.423673,0.0,0.0,19852
23050744,0.0,0.0,0.0,0.0,0.0,4200.38,840.076,183.910986,0.0,2.491429,-2.357551,0.0,0.0,19921


,name,ion,mass,mc,mc_low,mc_up,color,element,complex,isotope,charge
0,unranged0,un,0.0,0.0,0.0,400.0,#000000,[unranged],[0],[0],0


<div style="margin: 0 auto; padding: 20px;">

Load a pre saved range file if you want from cell below
</div>


In [6]:
button_r = widgets.Button(description='load range dataset')

@button_r.on_click
def open_file_on_click_r(b):
    global range_path
    folder_path = variables.last_directory
    script = '..//..//data_tools//run_dataset_path_qt.py'
    cmd = f"python {script} {folder_path}"
    result = subprocess.run(cmd, capture_output=True, text=True, shell=True)
    range_path = result.stdout.strip()
    variables.last_directory = range_path

button_r

Button(description='load range dataset', style=ButtonStyle())

In [7]:
if 'range_path' in globals():
    _, file_extension = os.path.splitext(range_path)
    if file_extension == '.rrng':
        variables.range_data = read_rrng(range_path)
    else:
        variables.range_data = pd.read_hdf(range_path, mode='r')
display(variables.range_data)

,name,ion,mass,mc,mc_low,mc_up,color,element,complex,isotope,charge
0,unranged0,un,0.0,0.0,0.0,400.0,#000000,[unranged],[0],[0],0


<div style="margin: 0 auto; padding: 20px;">

# Temporal crop

Select the data by drawing a rectangle over the experiment history. Experiment history is a 2D histogram of the time of flight of the ions versus sequence of evaporation. The experiment history is plotted by clicking on the button below te cell.
</div>


In [8]:
helper_temporal_crop.call_plot_crop_experiment(variables, pulse_mode.value)

Output()

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

# Spatial crop

Select the region of maximum concentration of Ions in the below plotted graph to utilize relevant data. To crop you can draw a circle over the filed desorption map. The field desorption map is a 2D histogram of the time of flight of the ions versus the position of the ions on the detector. The field desorption map is plotted by clicking on the button below the cell.
</div>

In [9]:
helper_special_crop.call_plot_crop_fdm(variables)

Output()

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Calculate pulses since the last event pulse and ions per pulse. The percentage of loss in ROI selection process will be printed.
</div>

In [10]:
# maximum value of the tdc counter
if tdc.value == 'pyccapt':
    if not (variables.data['delta_p'] != 0).any() or not (variables.data['multi'] != 0).any():
        if (variables.data['start_counter'] != 0).any():
            max_start_counter = max(variables.data['start_counter'])
            # if (variables.data['pulse_pi'] == 0).all() or (variables.data['ion_pp'] == 0).all():
            pulse_pi, ion_pp = data_loadcrop.calculate_ppi_and_ipp(variables.data, max_start_counter)
            # add two calculated array to the croped dataset
            variables.data['delta_p'] = pulse_pi.astype(np.uintc)
            variables.data['multi'] = ion_pp.astype(np.uintc)
    
    # save the cropped data
    print('tof Crop Loss {:.2f} %'.format((100 - (len(variables.data) / len(variables.data_backup)) * 100)))
#percentage of double event per pulse
ion_pp = variables.data['multi'].to_numpy()
print('percentage of multihit event per pulse', len(ion_pp[ion_pp != 1]) / float(len(ion_pp)) * 100)

Progress: 0% complete
Progress: 19% complete
Progress: 39% complete
Progress: 59% complete
Progress: 79% complete
Progress: 99% complete
tof Crop Loss 27.82 %
percentage of multihit event per pulse 3.4706176948758403


<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

In the next cell by changing the t<sub>0</sub> value you can correct the position of H<sub>1</sub> or any other known peak. this correction would be helpful for the position of the peaks in the m/c calibration process.
</div>

In [11]:
data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)
variables.data

The maximum possible time of flight is: 5010


,x (nm),y (nm),z (nm),mc (Da),mc_uc (Da),high_voltage (V),pulse,t (ns),t_c (ns),x_det (cm),y_det (cm),delta_p,multi,start_counter
0,0.0,0.0,0.0,0.0,0.0,3680.84,736.168,723.306402,0.0,0.943673,-2.778776,0,1,18197
1,0.0,0.0,0.0,0.0,0.0,3680.84,736.168,742.426506,0.0,2.432653,1.812245,39,1,18236
2,0.0,0.0,0.0,0.0,0.0,3680.84,736.168,721.633050,0.0,1.737143,0.300408,400,1,18636
3,0.0,0.0,0.0,0.0,0.0,3680.84,736.168,731.357694,0.0,0.212245,1.782857,37,1,18673
4,0.0,0.0,0.0,0.0,0.0,3680.84,736.168,726.989148,0.0,1.564082,-2.527347,405,1,19078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16636947,0.0,0.0,0.0,0.0,0.0,5234.12,1046.824,611.843328,0.0,1.469388,-2.383673,307,1,4887
16636948,0.0,0.0,0.0,0.0,0.0,5234.12,1046.824,610.910640,0.0,2.063673,-1.753469,86,1,4973
16636949,0.0,0.0,0.0,0.0,0.0,5234.12,1046.824,609.154992,0.0,-0.862041,-2.726531,15,1,4988
16636950,0.0,0.0,0.0,0.0,0.0,5234.12,1046.824,618.481872,0.0,1.596735,1.126531,406,1,5394


In [12]:
helper_t_0_tune.call_fine_tune_t_0(variables, flight_path_length, pulse_mode, t0)

Output()

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Remove the data with m/c greater than max m/c and x, y, t = 0. Also add the needed colums for calibration. The data types of the final cropped dataset is displayed below.

</div>

In [13]:
# add the columns in the dataset for x, y, z, mc, mc calibrated, and t calibrated
helper_data_loader.add_columns(variables, max_mc)
# save data temporarily
data_tools.save_data(variables.data, variables, hdf=True, epos=False, pos=False, csv=False, temp=True)
display(variables.data)
display(variables.data.dtypes)

The number of data over max_mc: 0
The number of data with having t, x, and y equal to zero is: 0


,x (nm),y (nm),z (nm),mc (Da),mc_uc (Da),high_voltage (V),pulse,t (ns),t_c (ns),x_det (cm),y_det (cm),delta_p,multi,start_counter
0,0.0,0.0,0.0,0.0,29.403503,3680.84,736.168,723.306402,0.0,0.943673,-2.778776,0,1,18197
1,0.0,0.0,0.0,0.0,30.936210,3680.84,736.168,742.426506,0.0,2.432653,1.812245,39,1,18236
2,0.0,0.0,0.0,0.0,30.556896,3680.84,736.168,721.633050,0.0,1.737143,0.300408,400,1,18636
3,0.0,0.0,0.0,0.0,31.408325,3680.84,736.168,731.357694,0.0,0.212245,1.782857,37,1,18673
4,0.0,0.0,0.0,0.0,29.671481,3680.84,736.168,726.989148,0.0,1.564082,-2.527347,405,1,19078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16636947,0.0,0.0,0.0,0.0,29.424786,5234.12,1046.824,611.843328,0.0,1.469388,-2.383673,307,1,4887
16636948,0.0,0.0,0.0,0.0,29.444546,5234.12,1046.824,610.910640,0.0,2.063673,-1.753469,86,1,4973
16636949,0.0,0.0,0.0,0.0,29.071973,5234.12,1046.824,609.154992,0.0,-0.862041,-2.726531,15,1,4988
16636950,0.0,0.0,0.0,0.0,31.084819,5234.12,1046.824,618.481872,0.0,1.596735,1.126531,406,1,5394


x (nm)              float64
y (nm)              float64
z (nm)              float64
mc (Da)             float64
mc_uc (Da)          float64
high_voltage (V)    float64
pulse               float64
t (ns)              float64
t_c (ns)            float64
x_det (cm)          float64
y_det (cm)          float64
delta_p              uint32
multi                uint32
start_counter        uint32
dtype: object

-------------------------------------------------------
-------------------------------------------------------

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## Time-of-Flight/Mass-to-charge Calibration
</div>

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
    
Below you can plot the ToF histogram of the dataset. You can select the peak range of the data you want to plot by drawing a rectangle over peak with holding left click. After that you should apply the voltage and bowl calibration. These three steps should be repeated until you see no improvement in the peak resolution. You can also save the calibration by clicking on the save button. The saved calibration will be used for the next steps.

</div>

In [14]:
# exctract needed data from Pandas data frame as an numpy array
data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)

The maximum possible time of flight is: 5010


In [15]:
helper_calibration.call_voltage_bowl_calibration(variables, det_diam.value, flight_path_length.value, pulse_mode.value)

In [16]:
variables.dld_t_calib_backup = np.copy(variables.dld_t_calib)
variables.mc_calib_backup = np.copy(variables.mc_calib)

In [17]:
helper_ion_list.call_ion_list(variables, selector='peak')

In [18]:
variables.data['mc (Da)'] = variables.mc_calib
variables.data['t_c (ns)'] = variables.dld_t_calib
# Remove negative mc
threshold = 0
mc_t = variables.data['mc (Da)'].to_numpy()
mc_t_mask = (mc_t <= threshold)
print('The number of ions with negative mc are:', len(mc_t_mask[mc_t_mask==True]))
variables.data.drop(np.where(mc_t_mask)[0], inplace=True)
variables.data.reset_index(inplace=True, drop=True)
# save data temporarily
data_tools.save_data(variables.data, variables, hdf=True, epos=False, pos=False, csv=False, temp=True)

The number of ions with negative mc are: 2489


In [19]:
helper_mc_plot.call_mc_plot(variables, selector='None')

Output()

-------------------------------------------------------
-------------------------------------------------------

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## 3D Reconstruction
After bowl and voltage calibration we are ready to calculate the 3d reconstruction. In this workflow we calculate the reconstructed x,y,z and then plot the 3d, heatmap, projection plots and mass-to-charge histogram.

</div>

In [20]:
# exctract needed data from Pandas data frame as a numpy array
data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)

display(variables.data)

The maximum possible time of flight is: 5010


,x (nm),y (nm),z (nm),mc (Da),mc_uc (Da),high_voltage (V),pulse,t (ns),t_c (ns),x_det (cm),y_det (cm),delta_p,multi,start_counter
0,0.0,0.0,0.0,27.023671,29.403503,3680.84,736.168,723.306402,656.477023,0.943673,-2.778776,0,1,18197
1,0.0,0.0,0.0,26.935078,30.936210,3680.84,736.168,742.426506,655.460971,2.432653,1.812245,39,1,18236
2,0.0,0.0,0.0,26.913140,30.556896,3680.84,736.168,721.633050,655.209121,1.737143,0.300408,400,1,18636
3,0.0,0.0,0.0,27.041236,31.408325,3680.84,736.168,731.357694,656.678267,0.212245,1.782857,37,1,18673
4,0.0,0.0,0.0,27.171525,29.671481,3680.84,736.168,726.989148,658.169010,1.564082,-2.527347,405,1,19078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16634458,0.0,0.0,0.0,27.188185,29.424786,5234.12,1046.824,611.843328,658.359375,1.469388,-2.383673,307,1,4887
16634459,0.0,0.0,0.0,26.975280,29.444546,5234.12,1046.824,610.910640,655.922243,2.063673,-1.753469,86,1,4973
16634460,0.0,0.0,0.0,27.040801,29.071973,5234.12,1046.824,609.154992,656.673286,-0.862041,-2.726531,15,1,4988
16634461,0.0,0.0,0.0,27.423863,31.084819,5234.12,1046.824,618.481872,661.046094,1.596735,1.126531,406,1,5394


<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

You have to select the main element in your sample from the from dropdown below.
</div>

In [4]:
element_selected = wd.density_field_selection()
display(element_selected)

Dropdown(description='Element', options=((1, 'H', 52.7, 0.0), (2, 'He', 28.7, 0.0), (3, 'Li', 46.3, 14.0), (4,…

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

In case that yopu face error about plotly library, like javascripts error. You have check your jupyter lab version is compatibale with the plotly extenstion. Sometimes running `jupyter lab build` command fix the proble.
</div>

In [22]:
helper_3d_reconstruction.call_x_y_z_calculation(variables, flight_path_length, element_selected=element_selected)

Output()

In [23]:
variables.data['x (nm)'] = variables.x
variables.data['y (nm)'] = variables.y
variables.data['z (nm)'] = variables.z
# save data temporarily
data_tools.save_data(variables.data, variables, hdf=True, epos=False, pos=False, csv=False, temp=True)

-------------------------------------------------------
-------------------------------------------------------

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## Ion Selection and Rangging

This tutorial outlines a comprehensive workflow for ion selection and organization. Users can choose ions using peak and element finders, manually add ions, customize ion colors, and create histograms. Histograms can be generated for selected ranges and areas, and figures can be saved. Additionally, users have the option to save both figures and data in CSV and HDF5 formats.

</div>

In [24]:
# exctract needed data from Pandas data frame as an numpy array
data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)

display(variables.data)

The maximum possible time of flight is: 5010


,x (nm),y (nm),z (nm),mc (Da),mc_uc (Da),high_voltage (V),pulse,t (ns),t_c (ns),x_det (cm),y_det (cm),delta_p,multi,start_counter
0,5.544648,-16.326975,3.173361,27.023671,29.403503,3680.84,736.168,723.306402,656.477023,0.943673,-2.778776,0,1,18197
1,14.248317,10.614518,3.376773,26.935078,30.936210,3680.84,736.168,742.426506,655.460971,2.432653,1.812245,39,1,18236
2,10.519766,1.819208,1.191354,26.913140,30.556896,3680.84,736.168,721.633050,655.209121,1.737143,0.300408,400,1,18636
3,1.284474,10.789582,1.234681,27.041236,31.408325,3680.84,736.168,731.357694,656.678267,0.212245,1.782857,37,1,18673
4,9.179021,-14.832072,3.250077,27.171525,29.671481,3680.84,736.168,726.989148,658.169010,1.564082,-2.527347,405,1,19078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16634458,12.327791,-19.998417,203.316053,27.188185,29.424786,5234.12,1046.824,611.843328,658.359375,1.469388,-2.383673,307,1,4887
16634459,17.361259,-14.751576,203.062549,26.975280,29.444546,5234.12,1046.824,610.910640,655.922243,2.063673,-1.753469,86,1,4973
16634460,-7.219228,-22.833542,203.482930,27.040801,29.071973,5234.12,1046.824,609.154992,656.673286,-0.862041,-2.726531,15,1,4988
16634461,13.695249,9.662292,201.255914,27.423863,31.084819,5234.12,1046.824,618.481872,661.046094,1.596735,1.126531,406,1,5394


In [25]:
display(variables.data.dtypes)

x (nm)              float64
y (nm)              float64
z (nm)              float64
mc (Da)             float64
mc_uc (Da)          float64
high_voltage (V)    float64
pulse               float64
t (ns)              float64
t_c (ns)            float64
x_det (cm)          float64
y_det (cm)          float64
delta_p              uint32
multi                uint32
start_counter        uint32
dtype: object

In [5]:
helper_ion_selection.call_ion_selection(variables)

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Now we can also show the color in the range dataframe

</div>

In [28]:
display(variables.range_data.style.applymap(ion_selection.display_color, subset=['color']))

,name,ion,mass,mc,mc_low,mc_up,color,element,complex,isotope,charge
0,H1,${}^{1}H^{+}$,1.010000,1.002499,0.847339,1.178480,#b2aa2d,['H'],[1],[1],1.000000
1,Al1,${}^{27}Al^{2+}$,13.490000,13.433483,13.253794,14.185893,#e7e0d1,['Al'],[1],[27],2.000000
2,Al1,${}^{27}Al^{+}$,26.980000,26.967215,26.542237,28.995128,#e7e0d1,['Al'],[1],[27],1.000000


In [29]:
variables.range_data.dtypes

name        object
ion         object
mass       float64
mc         float64
mc_low     float64
mc_up      float64
color       object
element     object
complex     object
isotope     object
charge     float64
dtype: object

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Save the range in the hdf5 and csv format.

</div>

In [30]:
interact_manual_range = interact_manual.options(manual_name="save range")
interact_manual_range(data_tools.save_range, variables=fixed(variables));

interactive(children=(Button(description='save range', style=ButtonStyle()), Output()), _dom_classes=('widget-…

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Save the cropped dataset. You can specify te output format from list below. The output formats are HDF5, EPOS, POS, ATO, and CSV. The output file will be saved in the same directory as the original dataset file in a new directory nammed load_crop.
</div>

In [31]:
interact_manual_data = interact_manual.options(manual_name="save data")
interact_manual_data(data_tools.save_data, data=fixed(variables.data), variables=fixed(variables),
                name=widgets.Text(value=variables.result_data_name),
                hdf=widgets.Dropdown(options=[('True', True), ('False', False)]),
                epos=widgets.Dropdown(options=[('False', False), ('True', True)]),
                pos=widgets.Dropdown(options=[('False', False), ('True', True)]),
                ato_6v=widgets.Dropdown(options=[('False', False), ('True', True)]),
                csv=widgets.Dropdown(options=[('False', False), ('True', True)]),
               temp=fixed(False));

interactive(children=(Text(value='2382_Jan-10-2025_15-12_NiC9_Al', continuous_update=False, description='name'…

-------------------------------------------------------
-------------------------------------------------------

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
    
# Visualization
    
</div>

In [32]:
variables.data

,x (nm),y (nm),z (nm),mc (Da),mc_uc (Da),high_voltage (V),pulse,t (ns),t_c (ns),x_det (cm),y_det (cm),delta_p,multi,start_counter
0,5.544648,-16.326975,3.173361,27.023671,29.403503,3680.84,736.168,723.306402,656.477023,0.943673,-2.778776,0,1,18197
1,14.248317,10.614518,3.376773,26.935078,30.936210,3680.84,736.168,742.426506,655.460971,2.432653,1.812245,39,1,18236
2,10.519766,1.819208,1.191354,26.913140,30.556896,3680.84,736.168,721.633050,655.209121,1.737143,0.300408,400,1,18636
3,1.284474,10.789582,1.234681,27.041236,31.408325,3680.84,736.168,731.357694,656.678267,0.212245,1.782857,37,1,18673
4,9.179021,-14.832072,3.250077,27.171525,29.671481,3680.84,736.168,726.989148,658.169010,1.564082,-2.527347,405,1,19078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16634458,12.327791,-19.998417,203.316053,27.188185,29.424786,5234.12,1046.824,611.843328,658.359375,1.469388,-2.383673,307,1,4887
16634459,17.361259,-14.751576,203.062549,26.975280,29.444546,5234.12,1046.824,610.910640,655.922243,2.063673,-1.753469,86,1,4973
16634460,-7.219228,-22.833542,203.482930,27.040801,29.071973,5234.12,1046.824,609.154992,656.673286,-0.862041,-2.726531,15,1,4988
16634461,13.695249,9.662292,201.255914,27.423863,31.084819,5234.12,1046.824,618.481872,661.046094,1.596735,1.126531,406,1,5394


# helper_visualization.call_visualization(variables)

In [33]:
data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)

The maximum possible time of flight is: 5010


In [10]:
helper_visualization.call_visualization(variables)

Output()

In [35]:
variables.plotly_3d_reconstruction.show()

In [ ]:
from IPython.display import display, HTML
display(HTML(variables.animation_detector_html))